# 案例：波士顿房价预测

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

## 1、获取数据
### 1.1 波士顿房价数据在sklearn中已经内置，可以通过load_boston()方法获得

In [3]:
boston = load_boston()

##### 特征含义
CRIM：城镇人均犯罪率。<br/>
ZN：住宅用地超过 25000 sq.ft. 的比例。<br/>
INDUS：城镇非零售商用土地的比例。<br/>
CHAS：查理斯河空变量（如果边界是河流，则为1；否则为0）。<br/>
NOX：一氧化氮浓度。<br/>
RM：住宅平均房间数。<br/>
AGE：1940 年之前建成的自用房屋比例。<br/>
DIS：到波士顿五个中心区域的加权距离。<br/>
RAD：辐射性公路的接近指数。<br/>
TAX：每 10000 美元的全值财产税率。<br/>
PTRATIO：城镇师生比例。<br/>
B：1000（Bk-0.63）^ 2，其中 Bk 指代城镇中黑人的比例。<br/>
LSTAT：人口中地位低下者的比例。<br/>
MEDV：自住房的平均房价，以千美元计。<br/>

In [4]:
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [5]:
print(boston)

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]]), 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
       35.4, 24.7, 31.6, 23.3, 19.6, 1

##### 取特征X和标签y

In [6]:
X = boston.data
y = boston.target

### 1.2 从文件读取
绝大多数情况，数据是存在文件中的，如excel。所以我们也可以从文件中读取数据。一般使用pandas读取。

In [10]:
import pandas as pd

In [12]:
df = pd.read_excel('data/boston.xls')
df

CRIM    ZN  INDUS  CHAS    NOX     RM    AGE     DIS  RAD  TAX  \
0     0.00632  18.0   2.31     0  0.538  6.575   65.2  4.0900    1  296   
1     0.02731   0.0   7.07     0  0.469  6.421   78.9  4.9671    2  242   
2     0.02729   0.0   7.07     0  0.469  7.185   61.1  4.9671    2  242   
3     0.03237   0.0   2.18     0  0.458  6.998   45.8  6.0622    3  222   
4     0.06905   0.0   2.18     0  0.458  7.147   54.2  6.0622    3  222   
5     0.02985   0.0   2.18     0  0.458  6.430   58.7  6.0622    3  222   
6     0.08829  12.5   7.87     0  0.524  6.012   66.6  5.5605    5  311   
7     0.14455  12.5   7.87     0  0.524  6.172   96.1  5.9505    5  311   
8     0.21124  12.5   7.87     0  0.524  5.631  100.0  6.0821    5  311   
9     0.17004  12.5   7.87     0  0.524  6.004   85.9  6.5921    5  311   
10    0.22489  12.5   7.87     0  0.524  6.377   94.3  6.3467    5  311   
11    0.11747  12.5   7.87     0  0.524  6.009   82.9  6.2267    5  311   
12    0.09378  12.5   7.87     0  0.524  5.889   39.0  5.4509    5  311   
13    0.62976   0.0   8.14     0  0.538  5.949   61.8  4.7075    4  307   
14    0.63796   0.0   8.14     0  0.538  6.096   84.5  4.4619    4  307   
15    0.62739   0.0   8.14     0  0.538  5.834   56.5  4.4986    4  307   
16    1.05393   0.0   8.14     0  0.538  5.935   29.3  4.4986    4  307   
17    0.78420   0.0   8.14     0  0.538  5.990   81.7  4.2579    4  307   
18    0.80271   0.0   8.14     0  0.538  5.456   36.6  3.7965    4  307   
19    0.72580   0.0   8.14     0  0.538  5.727   69.5  3.7965    4  307   
20    1.25179   0.0   8.14     0  0.538  5.570   98.1  3.7979    4  307   
21    0.85204   0.0   8.14     0  0.538  5.965   89.2  4.0123    4  307   
22    1.23247   0.0   8.14     0  0.538  6.142   91.7  3.9769    4  307   
23    0.98843   0.0   8.14     0  0.538  5.813  100.0  4.0952    4  307   
24    0.75026   0.0   8.14     0  0.538  5.924   94.1  4.3996    4  307   
25    0.84054   0.0   8.14     0  0.538  5.599   85.7  4.4546    4  307   
26    0.67191   0.0   8.14     0  0.538  5.813   90.3  4.6820    4  307   
27    0.95577   0.0   8.14     0  0.538  6.047   88.8  4.4534    4  307   
28    0.77299   0.0   8.14     0  0.538  6.495   94.4  4.4547    4  307   
29    1.00245   0.0   8.14     0  0.538  6.674   87.3  4.2390    4  307   
..        ...   ...    ...   ...    ...    ...    ...     ...  ...  ...   
476   4.87141   0.0  18.10     0  0.614  6.484   93.6  2.3053   24  666   
477  15.02340   0.0  18.10     0  0.614  5.304   97.3  2.1007   24  666   
478  10.23300   0.0  18.10     0  0.614  6.185   96.7  2.1705   24  666   
479  14.33370   0.0  18.10     0  0.614  6.229   88.0  1.9512   24  666   
480   5.82401   0.0  18.10     0  0.532  6.242   64.7  3.4242   24  666   
481   5.70818   0.0  18.10     0  0.532  6.750   74.9  3.3317   24  666   
482   5.73116   0.0  18.10     0  0.532  7.061   77.0  3.4106   24  666   
483   2.81838   0.0  18.10     0  0.532  5.762   40.3  4.0983   24  666   
484   2.37857   0.0  18.10     0  0.583  5.871   41.9  3.7240   24  666   
485   3.67367   0.0  18.10     0  0.583  6.312   51.9  3.9917   24  666   
486   5.69175   0.0  18.10     0  0.583  6.114   79.8  3.5459   24  666   
487   4.83567   0.0  18.10     0  0.583  5.905   53.2  3.1523   24  666   
488   0.15086   0.0  27.74     0  0.609  5.454   92.7  1.8209    4  711   
489   0.18337   0.0  27.74     0  0.609  5.414   98.3  1.7554    4  711   
490   0.20746   0.0  27.74     0  0.609  5.093   98.0  1.8226    4  711   
491   0.10574   0.0  27.74     0  0.609  5.983   98.8  1.8681    4  711   
492   0.11132   0.0  27.74     0  0.609  5.983   83.5  2.1099    4  711   
493   0.17331   0.0   9.69     0  0.585  5.707   54.0  2.3817    6  391   
494   0.27957   0.0   9.69     0  0.585  5.926   42.6  2.3817    6  391   
495   0.17899   0.0   9.69     0  0.585  5.670   28.8  2.7986    6  391   
496   0.28960   0.0   9.69     0  0.585  5.390   72.9  2.7986    6  391   
497   0.26838   0.0   9.69     0  

In [15]:
X = df[df.columns[0:-1]]
y = df[df.columns[-1]]


## 2、数据预处理(数据清洗)


我们获取的数据有可能存在下面的一些情况：
  - 缺少数据值
  - 含有错误的数据值，如年龄=200
  - 数据不一致，等级编码有的是“1，2，3”有的却是“A，B，C ”
  - 重复的记录值
  
$\color{red}{注意：本门课程关注的是机器学习算法，而波士顿房价数据也是清理过得，所以该部分不用写代码进行处理}$

## 3、数据分析与可视化

$\color{red}{注意：本门课程关注的是机器学习算法，不是数据分析，因此忽略数据分析与可视化部分}$

## 4、选择合适的机器学习模型

该问题是房价预测问题，线性回归能很好的应用于预测问题，因此我们选择使用线性回归模型

In [ ]:
model = linear_model.Ridge(alpha=0.1)
model.fit(X,y)
y_hat = model.predict(X)

我们如何选择参数alpha呢？

## 5、训练模型(使用交叉验证选择合适的参数)


In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [21]:
from sklearn.model_selection import GridSearchCV


In [22]:
ridge_model = linear_model.Ridge()
param = {'alpha':[0.01,0.03,0.05,0.07,0.1,0.5,0.8,1],'normalize':[True,False]}
gsearch = GridSearchCV(estimator=ridge_model,param_grid=param,cv=5,scoring='neg_mean_squared_error')
gsearch.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.01, 0.03, 0.05, 0.07, 0.1, 0.5, 0.8, 1],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [23]:
gsearch.best_params_,gsearch.best_score_

({'alpha': 0.01, 'normalize': True}, -22.186949592713958)

## 6、模型评价

In [27]:
final_model = linear_model.Ridge(alpha=0.01,normalize=True)
final_model.fit(X_train,y_train)
y_train_hat = final_model.predict(X_train)
y_test_hat = final_model.predict(X_test)
print("train-MSE=",mean_squared_error(y_train,y_train_hat))
print("test-MSE=",mean_squared_error(y_test,y_test_hat))

train-MSE= 20.31017854825943
test-MSE= 29.33714764884893


## 7、上线部署使用

1、模型保存

In [28]:
from sklearn.externals import joblib
joblib.dump(final_model,"house_train_model.m")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['house_train_model.m']

2、模型读取

In [31]:
load_model = joblib.load("house_train_model.m")

In [33]:
load_model.predict(X_test)

array([21.06146207, 30.76940069, 25.27764289,  8.85826012, 13.1679622 ,
       13.42874196, 28.40908273, 17.88860312, 35.52965554, 24.4624555 ,
       27.38148002, 17.86524752, 25.39484287, 31.601764  , 40.68945561,
        1.04294704, 22.48613919,  9.26382761, 15.47126053, 33.45246152,
       20.68268791, 22.12990584, 41.39077698, 25.25632265, 18.3549    ,
       27.70372119, 23.20470336, 15.34074708, 15.92935123, 32.77637859,
       16.15472678, 14.0826289 , 16.86915651, 30.92489386, 30.85566495,
       15.45326388, 20.32048959, 22.79388637, 24.12142587, 33.27537431,
       21.26839225, 12.11350321, 21.1816466 , 20.68839894, 17.36812778,
       11.85550438, 37.51917562, 13.5624638 ,  9.3764322 , 28.3551635 ,
       37.66089576, 37.1581289 , 21.64559063, 13.70527792, 31.50743511,
       17.09897449, 16.33767573, 30.47697741, 33.07152134, 30.57078175,
       31.254557  , 26.18456332, 15.25650601, 32.25078335, 24.48614704,
       21.36281395, 33.03159514, 28.4072058 , 11.36151172, 13.78